In [8]:
DATASET_DIR = '/Volumes/Manan/HandGestureDataset_SHREC2017'
INDEX_FILE = '../index.pkl'

## Prelude

In [9]:
from typing import Tuple
from pathlib import Path
import pickle

## Walkers

In [10]:
def getpcds(d: Path) -> list:
    files = d.iterdir()
    files = filter(lambda f: f.suffix == '.pcd', files)
    files = sorted(files, key=lambda f: int(f.name.split('_')[0]))
    files = map(lambda f: f.parts[-5:], files)
    files = map(lambda ps: '/' + '/'.join(ps), files)
    files = list(files)
    return files

In [11]:
getdirs = lambda d: filter(lambda e: e.is_dir(), d.iterdir())

In [12]:
def walk_dataset(p: str) -> dict:
    p = Path(p)
    D = dict()
    
    # gestures
    for g in getdirs(p):
        D[g.name] = dict()
        essais = list()
        
        # fingers
        for f in getdirs(g):
            D[g.name][f.name] = dict()
            
            # subjects
            for s in getdirs(f):
                D[g.name][f.name][s.name] = dict()
                
                # essais
                for e in getdirs(s):
                    pcds = getpcds(e)
                    D[g.name][f.name][s.name][e.name] = pcds
                    essais.append('/' + '/'.join(e.parts[-4:]))
                # !essais
            # !subjects
        #!fingers
        
        D[g.name]['essais'] = essais
    #!gestures
    
    return D

In [13]:
D = walk_dataset(DATASET_DIR)

In [14]:
with open(INDEX_FILE, 'wb') as f:
    pickle.dump(D, f)